# log6306-project

## Repositories

### Collections

- [bitnami/charts](https://github.com/bitnami/charts)
- [helm/charts](https://github.com/helm/charts)
- [k8s-at-home/charts](https://github.com/k8s-at-home/charts)

### Artifact Hub most viewed charts

From https://artifacthub.io/stats, monthly views, as of 2023-03-30

1. [kube-prometheus-stack](https://github.com/prometheus-community/helm-charts)
2. [ingress-nginx](https://github.com/kubernetes/ingress-nginx)
3. bitnami/postgresql
4. [cert-manager](https://github.com/cert-manager/cert-manager)
5. [argo-cd](https://github.com/argoproj/argo-helm)
6. bitnami/redis
7. [prometheus](https://github.com/prometheus-community/helm-charts)
8. [grafana](https://github.com/grafana/helm-charts)
9. [kubernetes-dashboard](https://github.com/kubernetes/dashboard)
10. [metrics-server](https://github.com/kubernetes-sigs/metrics-server)


## Extracting data

In [3]:
!git submodule update --init

In [1]:
import os
import re
import shutil
import subprocess
from contextlib import suppress
from pathlib import Path

from pydriller import Repository

PROJECT_DIR = Path('./projects/')
COMMITS_DIR = Path('./commits/')

MSG_MATCH_RE = re.compile(r'refactor', re.IGNORECASE)
FILES_MATCH_RE = re.compile(r'(templates\/.*|values).yaml')

# Cleanup
with suppress(Exception):
    shutil.rmtree(COMMITS_DIR)
os.mkdir(COMMITS_DIR)

for project in PROJECT_DIR.iterdir():
    if project.is_dir():
        print(f"Analyzing {project.name}...")
        # Output project directory
        diff_dir = COMMITS_DIR / project.name
        os.mkdir(diff_dir)
        commits = []
        for commit in Repository(project.as_posix()).traverse_commits():
            # Check commit message
            if MSG_MATCH_RE.search(commit.msg):
                # Check modified files
                git_diff_tree = subprocess.run(
                    ['git', 'diff-tree', '--no-commit-id',
                        '--name-only', '-r', '-m', commit.hash],
                    cwd=project.as_posix(),
                    capture_output=True)
                git_diff_tree_stdout = git_diff_tree.stdout.decode()
                if any(FILES_MATCH_RE.search(f) for f in git_diff_tree_stdout.splitlines()):
                    print(f"- Found commit {commit.hash}")
                    # Save diff
                    os.system(f'GIT_DIR={project.as_posix()}/.git '
                              f'git show -m -l 2000 {commit.hash} > {diff_dir / commit.hash}.diff')
                    # If this is a merge commit, remove children which are included
                    with open(f"{diff_dir / commit.hash}.diff", 'r') as f:
                        for prev_commits in commits:
                            if prev_commits.hash in f.read():
                                print(f"|- Removing {prev_commits.hash}")
                                os.remove(
                                    f"{diff_dir / prev_commits.hash}.diff")
                    commits.append(commit)


Analyzing prometheus-community...
- Found commit 875888ae2241bc00a30cccae5e521fd4ebd0cb4e
- Found commit 4bd1b180c578b50e44c18da35817ae591dd25b50
- Found commit 1af7e278a787036ef3e8dce7f388ca8ee6fa84cd
- Found commit 4e2511bd0c30bde538231d8896754c9d7fdd878e
- Found commit 01875d567cf238cebfca9d7bb49b8aa6ebb1f08f
- Found commit 1349fc508af7fb131fe77c617b4abae13948dc1a
- Found commit 2c14af93e1902751db5c7ca17182e0288e882b10
- Found commit df659245f42f19ff2704f4cca8947609dde2d755
- Found commit c9330ac11df9cb57249a8d5574b36bcecae097f5
Analyzing metrics-server...
Analyzing kubernetes-dashboard...
- Found commit 92a8491b99afa2cfb94dbe6f3410cadc42b0dc31
Analyzing cert-manager...
- Found commit 48cd58b23fb4753539a1f121b08356bf581cd232
- Found commit 8852b48068945ee88ed5741af9eea75c110cb63c
- Found commit 58754abf371090aaddeaaeecac58c390e7f63a74
- Found commit 7eacbd2eb917a38cdb3f1cd13e18d091ab86dc6c
- Found commit 54cc106f24ca15e81fd8f07233184e0e14d771da
- Found commit 1cdc5277c255a960bf319b6

## Analysis

### bitnami/charts

| Commit  | Description                                                                                                                                       |
| ------- | ------------------------------------------------------------------------------------------------------------------------------------------------- |
| 2f3891c | New Topology                                                                                                                                                |
| 6e64d41 | FP                                                                                                                                                |
| 7a2b828 | FP                                                                                                                                                |
| 7f24c45 | FP                                                                                                                                                |
| 8d1f38a | reorganize keys, tpl, labels                                                                                                                      |
| 9c53e01 | FP                                                                                                                                                |
| 028af48 | reorganize keys, tpl, add ingress                                                                                                                 |
| 31bcd6c | reorganize keys                                                                                                                                   |
| 40f825c | add security context                                                                                                                              |
| 43e7082 | refacto tpl, tpl, reorganize keys, probes                                                                                                         |
| 75aa09a | FP                                                                                                                                                |
| 91fa2c0 | reorganize keys, extraVol/init/env, tpl                                                                                                           |
| 155b46f | FP (new chart)                                                                                                                                    |
| 166ac3f | FP (helm 3 upgrade)                                                                                                                               |
| 786bd0c | unhardcode ports, probes                                                                                                                          |
| 5493b53 | more tpl, merge pods, probes, update notes.txt                                                                                                    |
| 5741a6d | more tpl, reorganize keys, add security context, unhardcode ports, add probes                                                                     |
| 47162ee | FP                                                                                                                                                |
| 66857b0 | reorganize keys                                                                                                                                   |
| a2dad36 | reorganize keys                                                                                                                                   |
| a2e6f7f | reorganize keys                                                                                                                                   |
| a55ce7b | new topology                                                                                                                                      |
| a762b7d | reorganize keys, update labels                                                                                                                    |
| ae8dac1 | reorder spec, add metrics, reorder env, update probes                                                                                             |
| b9cc743 | reorganize keys, update notes.txt                                                                                                                 |
| b65e552 | more tpl, quote things, update probes                                                                                                             |
| c4edd90 | rename volume, unhardcode ports                                                                                                                   |
| c401bdf | add resources to values, rename volume                                                                                                            |
| c25240b | update notes.txt, new topology, more tpl, unhardcode port, update probes, reorder volume, reorder spec                                            |
| ca74711 | reorganize keys, update notes.txt, unhardcode env, rename ports, ssl by default                                                                   |
| cf300d9 | if -> ternary func, update probes                                                                                                                 |
| d6b84ea | labels in tpl, rename container, update securityContext, unhardcode port, reorder values, unset resources, add ingress                            |
| ddf4a5e | add metrics, labels in tpl, default in values, use helm built-in vars, rename container, add readiness probe, merge configmaps, merge multiple if |
| e7da7f9 | FP                                                                                                                                                |
| fd08668 | reorganize keys                                                                                                                                   |


### helm/charts

### k8s-at-home/charts

### kube-prometheus-stack

### ingress-nginx

### cert-manager

### argo-cd

### prometheus

### grafana

### kubernetes-dashboard

### metrics-server

## Synthesis

| Technical Debt    | Refactoring Type            | Description                                          | Commits Count |
| ----------------- | --------------------------- | ---------------------------------------------------- | ------------- |
| Maintainability   | Named Template              | Add/Update named template to reduce code duplication | 10            |
| Maintainability   | Merge Resources             |                                                      | 2             |
| Modularity        | Support Multiple Topologies |                                                      | 2             |
| Modularity        | Unhardcode Value            |                                                      | 7             |
| Observability     | Expose Metrics              |                                                      | 2             |
| Observability     | Update Labels/Annotations   |                                                      | 4             |
| Observability     | Update Probes               |                                                      | 9             |
| Security          | Configure Security Context  |                                                      | 3             |
| Understandability | Rename Resource             |                                                      | 5             |
| Understandability | Reorder Spec                |                                                      | 2             |
| Understandability | Reorganize Keys             |                                                      | 13            |
